<a href="https://colab.research.google.com/github/SpyroRock/U-net-pixel-reconstruction/blob/master/U_net_pixel_reconstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/SpyroRock/U-net-pixel-reconstruction

Cloning into 'U-net-pixel-reconstruction'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 2047 (delta 7), reused 1 (delta 0), pack-reused 2033
Receiving objects: 100% (2047/2047), 250.52 MiB | 33.16 MiB/s, done.
Resolving deltas: 100% (1148/1148), done.
Checking out files: 100% (16/16), done.


In [2]:
%cd U-net-pixel-reconstruction

/content/U-net-pixel-reconstruction


In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 51.1MB 60kB/s 
     |████████████████████████████████| 307kB 54.6MB/s 
     |████████████████████████████████| 3.3MB 50.2MB/s 
     |████████████████████████████████| 573kB 42.1MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
     |████████████████████████████████| 3.3MB 48.0MB/s 
  Created wheel for keras-segmentation: filename=keras_segmentation-0.3.0-cp36-none-any.whl size=29072 sha256=6551703c706874ebcb1ad67d00f51e829e4e20c36d38b76c8022214ae7c83bc3
  Stored in directory: /root/.cache/pip/wheels/d0/bb/c0/6aad88b38f6e46db048bed4cccb904a5897055a8ab6fbd4dfc
Successfully built keras-segmentation
ERROR: xarray 0.15.1 has requirement setuptools>=41.2, but you'll have setuptools 39.1.0 which is incompatible.
ERROR: google-auth 1.25.0 has requirement setuptools>=40.3.0, but you'll have setuptools 39.1.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: 

In [1]:
%cd U-net-pixel-reconstruction

from unet_model_construction import get_unet
import keras
from keras.models import Model
from keras.layers import Input
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import backend as K
import numpy as np
from numpy import load, save
from sklearn.model_selection import train_test_split
import pickle
import os

/content/U-net-pixel-reconstruction


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarn

In [2]:
img_height = 256
img_width = 256

img_height_test = 256
img_width_test = 256

speckle_data = load('speckle_cluster_case2.npy')
print(speckle_data.shape)
speckle_labels = load('symbol_cluster_case2.npy')
print(speckle_labels.shape)

(180, 256, 256)
(180, 256, 256)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(speckle_data, speckle_labels, test_size=0.1, random_state=42)

X_train = X_train.reshape(-1, img_height, img_width, 1)
X_test = X_test.reshape(-1, img_height, img_width, 1)
input_shape = (img_height, img_width, 1)

y_train = y_train.reshape(-1, img_height_test, img_width_test, 1)
y_test = y_test.reshape(-1, img_height_test, img_width_test, 1)
input_shape_test = (img_height_test, img_width_test, 1)

print(X_train.shape)
print(y_train.shape)

(162, 256, 256, 1)
(162, 256, 256, 1)


In [ ]:
# checkpoint
#checkpoint_filepath = 'weights.{epoch:02d}-{accuracy:.2f}.hdf5'
checkpoint_filepath = '/content/U-net-pixel-reconstruction/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_filepath)


model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_filepath,
                                                               save_weights_only = True,
                                                               verbose=1)

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

reconstruction = get_unet(input_shape, n_filters = 32, dropout = 0.1, batchnorm = True)

reconstruction.fit(X_train, y_train, 
                   batch_size = 10,
                   epochs = 100,
                   verbose = 1,
                   validation_data = (X_test, y_test),    
                   callbacks = [model_checkpoint_callback])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 256, 256, 32) 128         conv2d_2[0][0]                   
__________________________________________________________________________________________________
activation_2 (Activation)       (None, 256, 256, 32) 0           batch_normalization_2[0][0]      
__________________________________________________________________________________________________
max_poolin

In [ ]:
reconstruction.load_weights(checkpoint_filepath)

score = reconstruction.evaluate(X_test, y_test, verbose = 0)

print('Test loss:', score[0])
print('Test acuracy:', score[1])

y_predicted = reconstruction.predict(X_test)

extract = Model(reconstruction.inputs, reconstruction.layers[-1].output) 
features = extract.predict(X_test)
print(features.shape)
save('features_data.npy', features)
save('features_predicted.npy', y_predicted)

reconstruction.save('reconstruction_model')

In [ ]:
import numpy as np 
from keras.models import load_model
import matplotlib.pyplot as plt
from numpy import load
import pickle
import tensorflow as tf
%matplotlib inline

#pickle_in = open('reconstruction.pkl', 'rb')
#reconstructor = pickle.load(pickle_in)

features_symbol = load('features_data.npy')
print(tf.constant(features_symbol))
print(features_symbol.shape)
features_symbol_predicted = load('features_predicted.npy')
print(tf.constant(features_symbol_predicted))
print(features_symbol_predicted.shape)

#print(features_symbol[10, :, :, 0])

#plt.imshow(features_symbol[5, :, :, 0], cmap='gray')
#plt.show()
print(np.min(features_symbol_predicted[17, :, :, 0]))
print(np.max(features_symbol_predicted[17, :, :, 0]))
plt.imshow(features_symbol_predicted[17, :, :, 0], cmap='gray')
plt.show()
